In [1]:
import json

def read_jsonl(file):
    # jsonl_data = []
    with open(file, 'r') as f:
        jsonl_data = json.load(f)
    #     for line in f:
    #         jsonl_data.append(line)

    return jsonl_data

props = read_jsonl('./query.json')
len(props)

11665

In [2]:
props[0]

{'property': 'http://www.wikidata.org/entity/P6',
 'propertyLabel': 'head of government',
 'propertyDescription': 'head of the executive power of this town, city, municipality, state, country, or other governmental body'}

In [3]:
from qdrant_client import QdrantClient
# qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance, for testing, CI/CD
# OR
client = QdrantClient(path="./db")  # Persists changes to disk, fast prototyping

In [ ]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example document and properties
document = "Your document text goes here."
properties = ["Property 1 text.", "Property 2 text.", ...]

# Convert text to vectors
doc_vector = model.encode(document, convert_to_tensor=True).numpy()
prop_vectors = model.encode(properties, convert_to_tensor=True).numpy()

# Setup Qdrant client
client = QdrantClient(host="localhost", port=6333)  # Adjust as needed

# Assuming you've already created a collection and indexed your properties
# For demonstration, we'll skip directly to querying

# Query for similar vectors
query_results = client.search(
    collection_name="your_collection_name",
    query_vector=doc_vector.tolist(),
    query_filter=None,  # Add filters if needed
    limit=5  # Number of top properties to find
)

# Process query results
for hit in query_results['hits']:
    print(f"Property ID: {hit['payload']['property_id']}, Score: {hit['score']}")
